In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

In [2]:
# 데이터 로드

train = pd.read_parquet('./train_fin.parquet')
train.head()

,id,base_date,day_of_week,base_hour,lane_count,road_rating,road_name,multi_linked,connect_code,maximum_speed_limit,...,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target,weekend,month
0,TRAIN_0000000,20220623,1,17,1,106,지방도1112호선,0,0,60.0,...,33.427747,126.662612,0,제3교래교,33.427749,126.662335,0,52.0,0,6
1,TRAIN_0000001,20220728,1,21,2,103,일반국도11호선,0,0,60.0,...,33.500730,126.529107,1,KAL사거리,33.504811,126.526240,0,30.0,0,7
2,TRAIN_0000002,20211010,4,7,2,103,일반국도16호선,0,0,80.0,...,33.279145,126.368598,0,상창육교,33.280072,126.362147,0,61.0,1,10
3,TRAIN_0000003,20220311,0,13,2,107,태평로,0,0,50.0,...,33.246081,126.567204,0,서현주택,33.245565,126.566228,0,20.0,0,3
4,TRAIN_0000004,20211005,6,8,2,103,일반국도12호선,0,0,80.0,...,33.462214,126.326551,0,애월입구,33.462677,126.330152,0,38.0,0,10


In [5]:
# 원핫인코딩 데이터 전처리 후에 아래 데이터 나누는 코드 실행

train = pd.concat([train, pd.get_dummies(train['base_hour'].apply(lambda x : int(x / 6)))] ,axis = 1).drop(['base_hour', 'month'], axis = 1)
train.rename(columns = {0 : 'h_0', 1 : 'h_1', 2 : 'h_2', 3 : 'h_3'}, inplace = True)

In [15]:
train.columns

Index(['id', 'base_date', 'day_of_week', 'lane_count', 'road_rating',
       'road_name', 'multi_linked', 'connect_code', 'maximum_speed_limit',
       'vehicle_restricted', 'weight_restricted', 'height_restricted',
       'road_type', 'start_node_name', 'start_latitude', 'start_longitude',
       'start_turn_restricted', 'end_node_name', 'end_latitude',
       'end_longitude', 'end_turn_restricted', 'target', 'weekend', 'h_0',
       'h_1', 'h_2', 'h_3'],
      dtype='object')

In [7]:
# 데이터 수정후 데이터 분리 진행 부탁드립니다

from sklearn.utils import shuffle

tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])

y_train = tr['target']
X_train = tr.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


y_test = te['target']
X_test = te.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

C:\Users\knzss\AppData\Local\Temp\ipykernel_17060\2061329703.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
C:\Users\knzss\AppData\Local\Temp\ipykernel_17060\2061329703.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])


(4426522, 16)
(4426522,)
(274695, 16)


## 저장된 데이터 셋이 공휴일 컬럼이 추가된 상태라 
## 기본 파라미터상태에서의 원핫인코딩 결과가 의미가 없을듯 합니다. 비교군이 달라져서요
## 기본 데이터에서의 기본 파라미터인데 데이터가 공휴일이 추가된 데이터라
## 아래 파라미터 하나만으로 원핫인코딩 테스트 진행하셔도 될듯합니다!!

In [16]:
# 원핫인코딩(시간) 테스트

min_num = 10
min_stack = 0
stop_num = 3

LR = lgb.LGBMRegressor(random_state = 42,
                       max_depth = 19,
                       num_leaves = 187,
                       n_estimators = 1000,
                       learning_rate = 0.6,
                      ).fit(X_train, y_train)

pred = LR.predict(X_test)
mae = mean_absolute_error(pred, y_test)

print('weekend_onehot : ', mae)


weekend_onehot :  4.390580733694007


# 스케일링 테스트

In [ ]:
# 데이터 로드

train = pd.read_parquet('./train_fin.parquet')
train.head()

In [ ]:
# 스케일링처리 코드



In [ ]:
# 데이터 수정후 데이터 분리 진행 부탁드립니다

from sklearn.utils import shuffle

tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])

y_train = tr['target']
X_train = tr.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


y_test = te['target']
X_test = te.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

In [ ]:
min_num = 10
min_stack = 0
stop_num = 3

LR = lgb.LGBMRegressor(random_state = 42,
                       max_depth = 19,
                       num_leaves = 187,
                       n_estimators = 1000,
                       learning_rate = 0.6,
                      ).fit(X_train, y_train)

pred = LR.predict(X_test)
mae = mean_absolute_error(pred, y_test)

# mae 값이 3.55와 근접하다면 for문으로 파라미터 테스트 간단하게 또 진행하도록 하겠습니다.
# 근접하다면 말해주세용
print('scaling : ', mae)

## 혹시 몰라서 for 문 파라미터 테스트 코드입니다

In [14]:
# 파라미터 테스트 범위 책정,
min_num_t = 1000
max_num_t = 1010

min_num = 10
min_stack = 0

# mae값이 3번째 커지면 for문이 종료됩니다
stop_num = 2


# 저는 파라미터 하나씩 주석을 풀면서 했습니다.
# 아래와 같이 하면 max_depth 파라미터 테스트
# max_depth를 주석처리하고 num_leaves만 주석을 해제하면 num_leaves 테스트

# for i in range(min_num_t, max_num_t + 1):
LR = lgb.LGBMRegressor(random_state = 42,

                       max_depth = 19,      # 20

                       num_leaves = 187,     # 200

                       n_estimators = 1000,   # 1000

                       learning_rate = 0.3,  # 0.6

#                            min_split_gain = (i / 100), # 0.0 mae값이 올라버림, 값이 오름에 따라
#                            min_child_weight = (i / 1000), # 영향 미비
#                            min_child_samples = i, # 20 미비
#                            colsample_bytree = (i / 10), # 1.0이 젤 좋음
#                            reg_lambda = (i / 100),
                      ).fit(X_train, y_train)

pred = LR.predict(X_test)
mae = mean_absolute_error(pred, y_test)

print('파라미터 값  : ', i, ', mae : ', mae)

if min_num > mae:
    min_stack = 0
    min_num = mae
elif min_stack != stop_num:
    min_stack += 1
    min_num = mae
elif min_stack == stop_num:
    break

파라미터 값  :  1002 , mae :  4.390580643260661


SyntaxError: 'break' outside loop (2641391234.py, line 46)